In [1]:
import numpy as np
import pandas as pd


<p>گسسته‌سازی (Discretization) به معنی تقسیم مقدار پیوسته‌ی یک متغیر به بازه‌های مشخص است. معمولاً این بازه‌ها ترتیب مشخصی دارند. مثلاً، سن که به شکل عدد بیان شده است را می‌توان گسسته کرد؛ اگر عملیات گسسته‌سازی روی سن انجام شود، می‌توانیم بازه‌های سنی داشته باشیم. مثلاً: «کودک»، «نوجوان» و «بزرگسال».

برای گسسته‌سازی روش یکتایی وجود ندارد. برای این کار، کتابخانه‌ی پانداز تابع cut را به همین منظور پیشنهاد داده‌است؛ اما ما می‌توانیم بدون استفاده از تابع cut نیز گسسته‌سازی انجام دهیم.

محتوای این درسنامه در دو بخش ارائه می‌شود. در بخش اول گسسته‌سازی را بدون استفاده از تابع cut تمرین می‌کنیم و در بخش دوم، نحوه استفاده از تابع cut را می‌آموزیم.</p>

<h3>گسسته‌سازی بدون cut</h3>
<p>به‌عنوان مثال، فرض کنید در حال تحلیل داده‌های مربوط به اورژانس بیمارستان‌های شهر اصفهان در شهریور ماه ۱۴۰۰ هستید. می‌خواهید بررسی کنید که تعداد دفعات اعزام آمبولانس چگونه در ساعات شبانه‌روز توزیع شده است؛ به‌ عبارت دیگر، در هر ساعتی از روز چند بار آمبولانس‌های اورژانس شهر اصفهان به محل اعزام شده‌اند.

با اجرای سلول زیر، دیتافریم df ایجاد می‌شود. ستون time، زمانی که یک آمبولانس اعزام شده‌ را نشان می‌دهد و ستون area‍، منطقه‌ای را نشان می‌دهد که آمبولانس به آن‌جا اعزام شده است. به‌عنوان مثال، سطر اول اطلاعات آمبولانسی را به‌دست می‌دهد که در ساعت ۱۲:۲۴:۰۰ به منطقه‌ی ۲ اصفهان اعزام شده است.</p>

In [2]:
from datetime import datetime
times = [datetime(2021,8,24, 12, 24), datetime(2021,9,2, 23, 49), datetime(2021,9,10, 2, 15), datetime(2021,8,30, 16, 35),\
         datetime(2021,9,4, 15, 40), datetime(2021,9,14, 18, 0), datetime(2021,9,12, 21, 0), datetime(2021,9,17, 20, 12),\
         datetime(2021,9,1, 7, 4), datetime(2021,9,10, 14, 20)]

df = pd.DataFrame()
df['time'] = times
df['area'] = [2, 4, 10, 4, 3, 1, 7, 5, 2, 8]
df


,time,area
0,2021-08-24 12:24:00,2
1,2021-09-02 23:49:00,4
2,2021-09-10 02:15:00,10
3,2021-08-30 16:35:00,4
4,2021-09-04 15:40:00,3
5,2021-09-14 18:00:00,1
6,2021-09-12 21:00:00,7
7,2021-09-17 20:12:00,5
8,2021-09-01 07:04:00,2
9,2021-09-10 14:20:00,8


<p>برای تحلیل آن‌چه در بالا گفته شده بود (دفعات اعزام در ساعات یک شبانه‌روز) دو راه وجود دارد.

    می‌توانیم ستون time را تغییر ندهیم و فرضاً محاسبه کنیم، در ساعت ۲:۱۵:۰۰ چند مرتبه آمبولانس اعزام شده است. (به نظر شما، چند سطر وجود دارد که ساعت اعزام آمبولانس در آن‌ها برابر ۲:۱۵:۰۰ باشد؟!)
    راه‌حل دیگر این است که ساعت‌های نوشته شده در ستون time را گسسته کنیم. می‌توانیم ۲۴ ساعت شبانه‌روز را طبق جدول زیر به ۵ قسمت تقسیم کنیم:
</p>

In [3]:
from datetime import time
dawn = time(hour=7,minute=0,second=0)
morning = time(hour=11,minute=0,second=0)
noon = time(hour=15,minute=0,second=0)
afternoon = time(hour=19,minute=0,second=0)

def discretize_time (time) :
  if time.time() < dawn :
    return "dawn"
  elif time.time() < morning :
    return "morning"
  elif time.time() < noon :
    return "noon"
  elif time.time() < afternoon :
    return "afternoon"
  else : return "night"

df['time'] = df.time.apply(discretize_time)
df


,time,area
0,noon,2
1,night,4
2,dawn,10
3,afternoon,4
4,afternoon,3
5,afternoon,1
6,night,7
7,night,5
8,morning,2
9,noon,8


<p>
Plain text

پس از اعمال این تغییر، تحلیل داده بسیار راحت‌تر است. اگر قبلاً هر سطر یک time مختص به خود داشت، در مجموعه‌داده‌ی جدید، سطر‌هایی وجود دارند که مقدار ستون time یکسانی دارند.

حال به نظر شما، فهم کدام یک از این دو جدول راحت‌تر است؟! با نگاه کردن به کدام جدول، نسبت به زمان اعزام آمبولانس‌های اصفهان شهود بهتری پیدا کردید؟</p>

<h3>معرفی تابع cut🔗</h3>
<p>روش‌های مختلفی برای گسسته‌سازی وجود دارد. یکی از راه‌هایی که پانداز برای گسسته‌سازی معرفی می‌کند، استفاده از تابع cut است. این تابع با دریافت ساختمان‌‌داده‌ای شبیه به آرایه (که باید یک‌بعدی باشد) و مرز‌های تقسیم‌بندی، آرایه‌ ورودی را گسسته‌سازی می‌کند. در فایل نت‌بوک درسنامه، جزئیات تابع و جدول پارامترهایش را اضافه کرده‌ایم.

تصور کنید نام و سن ۸ نفر را در اختیار دارید.</p>

In [4]:
df = pd.DataFrame()
df['name'] = ["helma","sajede","ali","hossein","mohsen","haniye","hasan","sajjad"]
df['age'] = [5, 15, 12, 60, 35, 17, 71, 23]
df


,name,age
0,helma,5
1,sajede,15
2,ali,12
3,hossein,60
4,mohsen,35
5,haniye,17
6,hasan,71
7,sajjad,23


In [5]:
bins = [1, 11, 18, 35, 50, 100]
labels = ["child", "teenager", "young", "middleaged", "old"]
df['age'] = pd.cut(df.age, bins, labels=labels)
df


,name,age
0,helma,child
1,sajede,teenager
2,ali,teenager
3,hossein,old
4,mohsen,young
5,haniye,teenager
6,hasan,old
7,sajjad,young


<p>گسسته‌سازی عملیاتی است که بیشتر در گام پیش‌پردازش یادگیری ماشین انجام می‌شود؛ بنابراین استفاده به موقع از گسسته‌سازی، به بهبود عملکرد مدل‌های یادگیری ماشین کمک شایانی می‌کند. اما گسسته‌سازی در تحلیل داده هم بی‌تاثیر نیست.

    ممکن است مقادیر یک ستون دارای نویز باشد. با گسسته‌سازی می‌توانیم اثر نویز را کاهش دهیم. منظور از نویز، تغییرات کوچکی است که روی اطلاعات اعمال شده است.
    بسته به صورت‌مسئله‌ای که به دنبال حل آن هستیم، گسسته‌سازی می‌تواند مفید باشد. مثلا در مسئله بازاریابی، سن افراد می‌تواند گسسته‌ شود. فروشگاه آمازون برای پیشنهاد کالا می‌تواند سلیقه‌‌ی افراد ۲۲ ساله را با افراد ۲۳، ۲۴ یا حتی ۲۰ ساله یکسان تصور کند. پس در این مثال با شرط اینکه می‌خواهیم فقط از سن برای پیشنهاد کالا استفاده کنیم، می‌توان سنین ۲۰ تا ۲۵ را در یک دسته قرار داد!
</p>